# Визуализация данных квартир города Днепр

Этот ноутбук содержит интерактивную визуализацию данных о квартирах в городе Днепр с использованием тепловых карт и маркеров цен.

## Содержание:
1. Загрузка и исследование данных
2. Предобработка и очистка данных
3. Статистический анализ
4. Интерактивная карта с тепловой картой плотности квартир
5. Маркеры со средними ценами по районам
6. Дополнительные визуализации с Plotly

## 1. Импорт необходимых библиотек

In [1]:
# Импорт основных библиотек для работы с данными
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Карты
import folium
from folium.plugins import HeatMap, MarkerCluster
from folium import plugins

# Геокодирование
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Настройка стиля
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Все библиотеки успешно импортированы!")

✓ Все библиотеки успешно импортированы!


## 2. Загрузка и первичный анализ данных

In [2]:
# Загрузка данных
df = pd.read_csv('/home/redmoon/Documents/Dev/appartment_scraper/cache/apartments_cache.csv')

# Общая информация о датасете
print(f"Размер датасета: {df.shape[0]} строк, {df.shape[1]} столбцов")
print(f"\n{'='*60}")
print("Первые 5 записей:")
print(df.head())

print(f"\n{'='*60}")
print("Информация о типах данных:")
print(df.info())

print(f"\n{'='*60}")
print("Статистика по числовым полям:")
print(df.describe())

Размер датасета: 1054 строк, 19 столбцов

Первые 5 записей:
     post_id                                               name    price  \
0  898655533  Аренда двухкомнатной квартиры Гагарина район П...  14000.0   
1  900827364  Аренда 3к квартиры в центре города, Конисского...  13000.0   
2  898811943  Сдам 2 к -кв Жк Зоряний первая сдача автономно...  17000.0   
3  899818032                        Здам 4х комнатную  квартиру  45000.0   
4  899686221              Сдам 3ком.кв  106кв.м ж.к.жуковского.  45000.0   

  currency                         location  \
0      UAH                 Дніпро, Соборний   
1      UAH           Дніпро, Шевченківський   
2      UAH           Дніпро, Шевченківський   
3      UAH  Дніпро, Амур-Нижньодніпровський   
4      UAH  Дніпро, Амур-Нижньодніпровський   

                                         description  contact_phone  \
0  Сдам двухкомнатную квартиру на первом очень вы...            NaN   
1  Аренда #3к квартиры\nСорочинский переулок #сор...      

## 3. Предобработка и очистка данных

In [3]:
# Очистка данных
# Удаляем записи с пропущенными ценами
df_clean = df.dropna(subset=['price', 'district']).copy()

# Фильтруем аномальные цены (меньше 1000 или больше 100000)
df_clean = df_clean[(df_clean['price'] >= 1000) & (df_clean['price'] <= 100000)]

# Заполняем пропущенные значения в комнатах
df_clean['rooms'] = df_clean['rooms'].fillna(1)

# Проверяем районы
print("Уникальные районы:")
print(df_clean['district'].value_counts())
print(f"\nВсего записей после очистки: {len(df_clean)}")

Уникальные районы:
district
Соборний                   273
Центральний                248
Індустріальний             166
Шевченківський             145
Чечелівський                77
Амур-Нижньодніпровський     70
Новокодацький               49
Самарський                  23
Name: count, dtype: int64

Всего записей после очистки: 1051


### Геокодирование районов Днепра

Создадим координаты для каждого района города Днепр

In [4]:
# Координаты центров районов Днепра (приблизительные)
district_coords = {
    'Соборний': (48.4647, 35.0462),
    'Шевченківський': (48.4500, 35.0400),
    'Амур-Нижньодніпровський': (48.4200, 35.1000),
    'Новокодацький': (48.5000, 35.1200),
    'Самарський': (48.5200, 35.0800),
    'Індустріальний': (48.5000, 34.9800),
    'Чечелівський': (48.4800, 35.0200),
    'Центральний': (48.4647, 35.0462)  # используем координаты центра города
}

# Добавляем координаты в датафрейм
df_clean['lat'] = df_clean['district'].map(lambda x: district_coords.get(x, (48.4647, 35.0462))[0])
df_clean['lon'] = df_clean['district'].map(lambda x: district_coords.get(x, (48.4647, 35.0462))[1])

# Добавляем небольшой случайный разброс для лучшей визуализации
np.random.seed(42)
df_clean['lat'] = df_clean['lat'] + np.random.normal(0, 0.01, len(df_clean))
df_clean['lon'] = df_clean['lon'] + np.random.normal(0, 0.01, len(df_clean))

print("✓ Координаты добавлены!")

✓ Координаты добавлены!


## 4. Статистический анализ данных

In [8]:
# Агрегированная статистика по районам
district_stats = df_clean.groupby('district').agg({
    'price': ['mean', 'median', 'count', 'std'],
    'total_area': 'mean',
    'rooms': 'mean'
}).round(2)

district_stats.columns = ['Средняя цена', 'Медианная цена', 'Количество', 'Ст. отклонение', 'Средняя площадь', 'Средн. комнат']
district_stats = district_stats.sort_values('Количество', ascending=False)

print("Статистика по районам:")
print(district_stats)

# Сохраняем для дальнейшего использования
district_stats_dict = district_stats.to_dict('index')

Статистика по районам:
                         Средняя цена  Медианная цена  Количество  \
district                                                            
Соборний                     17592.23         15000.0         273   
Центральний                  14726.50         13000.0         248   
Індустріальний               11342.76         11000.0         166   
Шевченківський               13800.72         12000.0         145   
Чечелівський                 11861.03         11000.0          77   
Амур-Нижньодніпровський      12432.86         10500.0          70   
Новокодацький                 9093.88          9000.0          49   
Самарський                    8904.35          8000.0          23   

                         Ст. отклонение  Средняя площадь  Средн. комнат  
district                                                                 
Соборний                       10083.39            51.96           1.76  
Центральний                     6914.83            48.01        

### Визуализация распределения цен с Plotly

In [13]:
# Интерактивный box plot цен по районам
fig1 = px.box(df_clean, x='district', y='price', 
              title='Распределение цен по районам',
              labels={'district': 'Район', 'price': 'Цена (грн)'},
              color='district',
              height=500)

fig1.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,
    template='plotly_white'
)

fig1.show()

# Гистограмма распределения цен
fig2 = px.histogram(df_clean, x='price', nbins=50,
                    title='Распределение цен на квартиры',
                    labels={'price': 'Цена (грн)', 'count': 'Количество'},
                    color_discrete_sequence=['#636EFA'])

fig2.update_layout(template='plotly_white', height=400)
fig2.show()

In [6]:
# Средние цены по районам - интерактивный bar chart
avg_prices = df_clean.groupby('district')['price'].mean().sort_values(ascending=False).reset_index()

fig3 = px.bar(avg_prices, x='district', y='price',
              title='Средние цены на аренду по районам',
              labels={'district': 'Район', 'price': 'Средняя цена (грн)'},
              color='price',
              color_continuous_scale='Viridis',
              height=500)

fig3.update_layout(
    xaxis_tickangle=-45,
    template='plotly_white'
)

fig3.show()

## 5. Создание интерактивной карты Днепра с Folium

### Карта с тепловой картой плотности квартир и маркерами средних цен

In [7]:
# Создание базовой карты с центром в Днепре
dnipro_center = [48.4647, 35.0462]

# Создаем карту
m = folium.Map(
    location=dnipro_center,
    zoom_start=12,
    tiles='OpenStreetMap',
    control_scale=True
)

# Добавляем альтернативные слои карты
folium.TileLayer('CartoDB positron', name='CartoDB Positron').add_to(m)
folium.TileLayer('CartoDB dark_matter', name='CartoDB Dark').add_to(m)

print("✓ Базовая карта создана!")

✓ Базовая карта создана!


In [9]:
# Подготовка данных для тепловой карты (количество квартир)
heat_data = [[row['lat'], row['lon']] for idx, row in df_clean.iterrows()]

# Создание тепловой карты плотности
HeatMap(
    heat_data,
    name='Тепловая карта плотности квартир',
    min_opacity=0.3,
    max_opacity=0.8,
    radius=15,
    blur=20,
    gradient={0.4: 'blue', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'}
).add_to(m)

print("✓ Тепловая карта добавлена!")

✓ Тепловая карта добавлена!


In [10]:
# Добавление маркеров со средними ценами по районам
# Группируем данные по районам для маркеров
district_summary = df_clean.groupby('district').agg({
    'price': 'mean',
    'lat': 'mean',
    'lon': 'mean',
    'post_id': 'count'  # количество объявлений
}).reset_index()

district_summary.columns = ['district', 'avg_price', 'lat', 'lon', 'count']

# Создаем feature group для маркеров
marker_cluster = plugins.MarkerCluster(name='Средние цены по районам').add_to(m)

# Добавляем маркеры для каждого района
for idx, row in district_summary.iterrows():
    # Определяем цвет маркера на основе цены
    if row['avg_price'] < 10000:
        color = 'green'
    elif row['avg_price'] < 20000:
        color = 'orange'
    else:
        color = 'red'
    
    # HTML для popup
    popup_html = f"""
    <div style="font-family: Arial; font-size: 12px; width: 200px;">
        <h4 style="color: #333; margin: 0 0 10px 0;">{row['district']}</h4>
        <hr style="margin: 5px 0;">
        <b>Средняя цена:</b> {row['avg_price']:.0f} грн<br>
        <b>Количество:</b> {row['count']} объявлений<br>
    </div>
    """
    
    # Создаем маркер
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html, max_width=250),
        tooltip=f"{row['district']}: {row['avg_price']:.0f} грн",
        icon=folium.Icon(color=color, icon='home', prefix='fa')
    ).add_to(marker_cluster)

print("✓ Маркеры с ценами добавлены!")

✓ Маркеры с ценами добавлены!


In [11]:
# Добавляем легенду
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 200px; height: auto; 
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; border-radius: 5px; padding: 10px">
    <h4 style="margin: 0 0 10px 0;">Легенда</h4>
    <p style="margin: 5px 0;"><i class="fa fa-map-marker" style="color:green"></i> < 10,000 грн</p>
    <p style="margin: 5px 0;"><i class="fa fa-map-marker" style="color:orange"></i> 10,000 - 20,000 грн</p>
    <p style="margin: 5px 0;"><i class="fa fa-map-marker" style="color:red"></i> > 20,000 грн</p>
    <hr style="margin: 10px 0;">
    <p style="margin: 5px 0; font-size: 12px;"><b>Тепловая карта:</b> плотность квартир</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Добавляем контроль слоев
folium.LayerControl(position='topright').add_to(m)

print("✓ Легенда и контроль слоев добавлены!")

✓ Легенда и контроль слоев добавлены!


In [12]:
# Сохраняем карту
map_path = '/home/redmoon/Documents/Dev/appartment_scraper/charts/dnipro_apartments_map.html'
m.save(map_path)
print(f"✓ Карта сохранена: {map_path}")

# Отображаем карту
m

✓ Карта сохранена: /home/redmoon/Documents/Dev/appartment_scraper/charts/dnipro_apartments_map.html


## 6. Альтернативная визуализация с Plotly Mapbox

Интерактивная карта с точками, размер которых зависит от количества квартир, а цвет от средней цены

In [14]:
# Plotly scatter mapbox с данными по районам
fig_map = px.scatter_mapbox(
    district_summary,
    lat='lat',
    lon='lon',
    size='count',
    color='avg_price',
    hover_name='district',
    hover_data={
        'avg_price': ':,.0f',
        'count': True,
        'lat': False,
        'lon': False
    },
    color_continuous_scale='Viridis',
    size_max=30,
    zoom=11,
    title='Интерактивная карта средних цен и количества квартир в Днепре',
    labels={
        'avg_price': 'Средняя цена (грн)',
        'count': 'Количество объявлений'
    },
    height=700
)

fig_map.update_layout(
    mapbox_style='open-street-map',
    mapbox_center={'lat': 48.4647, 'lon': 35.0462},
    margin={'r': 0, 't': 50, 'l': 0, 'b': 0}
)

fig_map.show()

## 7. Дополнительные продвинутые визуализации

In [15]:
# Sunburst диаграмма: иерархия цен по районам и количеству комнат
df_sunburst = df_clean.groupby(['district', 'rooms']).agg({
    'price': 'mean',
    'post_id': 'count'
}).reset_index()

df_sunburst['rooms_str'] = df_sunburst['rooms'].astype(int).astype(str) + ' комн.'

fig_sunburst = px.sunburst(
    df_sunburst,
    path=['district', 'rooms_str'],
    values='post_id',
    color='price',
    color_continuous_scale='RdYlGn_r',
    title='Распределение квартир по районам и количеству комнат',
    labels={'post_id': 'Количество', 'price': 'Средняя цена (грн)'},
    height=700
)

fig_sunburst.update_layout(template='plotly_white')
fig_sunburst.show()

In [16]:
# 3D scatter plot: цена vs площадь vs количество комнат
fig_3d = px.scatter_3d(
    df_clean.sample(min(1000, len(df_clean))),  # берем выборку для производительности
    x='total_area',
    y='rooms',
    z='price',
    color='district',
    title='3D визуализация: Площадь, Количество комнат и Цена',
    labels={
        'total_area': 'Площадь (м²)',
        'rooms': 'Количество комнат',
        'price': 'Цена (грн)',
        'district': 'Район'
    },
    height=700,
    opacity=0.7
)

fig_3d.update_layout(template='plotly_white')
fig_3d.show()

In [17]:
# Violin plot - распределение цен с подробностями
fig_violin = px.violin(
    df_clean,
    x='district',
    y='price',
    color='district',
    box=True,
    points='outliers',
    title='Детальное распределение цен по районам (Violin Plot)',
    labels={'district': 'Район', 'price': 'Цена (грн)'},
    height=600
)

fig_violin.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,
    template='plotly_white'
)

fig_violin.show()

In [18]:
# Treemap - визуализация количества и цен
df_treemap = df_clean.groupby(['district', 'rooms']).agg({
    'price': 'mean',
    'post_id': 'count'
}).reset_index()

df_treemap['rooms_label'] = df_treemap['rooms'].astype(int).astype(str) + ' комн.'
df_treemap['label'] = df_treemap['district'] + '<br>' + df_treemap['rooms_label']

fig_treemap = px.treemap(
    df_treemap,
    path=[px.Constant("Днепр"), 'district', 'rooms_label'],
    values='post_id',
    color='price',
    color_continuous_scale='Turbo',
    title='Treemap: Распределение квартир по районам и комнатам',
    labels={'post_id': 'Количество', 'price': 'Средняя цена'},
    height=700
)

fig_treemap.update_traces(textinfo='label+value')
fig_treemap.update_layout(template='plotly_white')
fig_treemap.show()

## 8. Корреляционный анализ

In [19]:
# Матрица корреляции
numeric_cols = ['price', 'total_area', 'floor', 'total_floors', 'rooms']
correlation_matrix = df_clean[numeric_cols].corr()

# Интерактивная heatmap корреляций
fig_corr = px.imshow(
    correlation_matrix,
    text_auto='.2f',
    aspect='auto',
    color_continuous_scale='RdBu_r',
    title='Матрица корреляции характеристик квартир',
    labels=dict(x='Параметры', y='Параметры', color='Корреляция'),
    height=600
)

fig_corr.update_layout(template='plotly_white')
fig_corr.show()

## 9. Итоговая сводка

Краткая статистика по визуализациям

In [20]:
print("="*70)
print("ИТОГОВАЯ СТАТИСТИКА ПО КВАРТИРАМ В ДНЕПРЕ")
print("="*70)

print(f"\n📊 Общее количество объявлений: {len(df_clean)}")
print(f"💰 Средняя цена аренды: {df_clean['price'].mean():.2f} грн")
print(f"📍 Медианная цена: {df_clean['price'].median():.2f} грн")
print(f"📈 Самый дорогой район: {district_summary.loc[district_summary['avg_price'].idxmax(), 'district']}")
print(f"📉 Самый дешевый район: {district_summary.loc[district_summary['avg_price'].idxmin(), 'district']}")
print(f"🏘️  Район с наибольшим количеством предложений: {district_summary.loc[district_summary['count'].idxmax(), 'district']}")

print(f"\n{'='*70}")
print("📁 СОЗДАННЫЕ ФАЙЛЫ:")
print(f"   - Интерактивная карта: {map_path}")
print(f"\n{'='*70}")
print("✅ Визуализация завершена!")

ИТОГОВАЯ СТАТИСТИКА ПО КВАРТИРАМ В ДНЕПРЕ

📊 Общее количество объявлений: 1051
💰 Средняя цена аренды: 14056.00 грн
📍 Медианная цена: 12000.00 грн
📈 Самый дорогой район: Соборний
📉 Самый дешевый район: Самарський
🏘️  Район с наибольшим количеством предложений: Соборний

📁 СОЗДАННЫЕ ФАЙЛЫ:
   - Интерактивная карта: /home/redmoon/Documents/Dev/appartment_scraper/charts/dnipro_apartments_map.html

✅ Визуализация завершена!
